In [3]:
import formulas
import utilities
import pandas as pd
import os
import re


def create_formulas_content(website_creator_dir, docs_dir):

    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv' 
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    
    df_by_year =  (
        formulas.get_formulas_by_year_df(formula_file_path))
    df_cumulative = (
        formulas.get_formulas_by_year_cumulative_df(
            formula_file_path, order_file_path))
    sort_orders_df = pd.read_csv(order_file_path)
    df_formulas = pd.concat([df_by_year, df_cumulative])
    formula_sheet_list = formulas.get_formulas_on_formula_sheet(
        formula_file_path)
    
    dirs_df = (
        df_formulas[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code']].drop_duplicates())
    file_paths_df = (
        df_formulas[['State', 'Formula sub category 1', 
                             'Formula sub category 2',
                             'Subject code', 'Category']].drop_duplicates())
    
    utilities.create_sub_directories_from_df(base_dir = docs_dir, 
                                             sub_paths_df = dirs_df)
    
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=get_formula_display_string, 
                           sort_orders_df = sort_orders_df,
                           df_formulas = df_formulas, 
                           formula_sheet_list = formula_sheet_list, 
                           cols_to_highlight_if_in_formula_sheet = [
                               'Formula_1', 'Formula_2']
                          )


if __name__ == '__main__':

    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    utilities.delete_directory_if_it_exists(docs_dir)
    create_formulas_content(website_creator_dir, docs_dir)


    # df_calculus = formulas.df_calculus_summary(df_formulas)
    # styler_calculus = formulas.styler_calculus_summary(
    #     df_calculus, formula_sheet_list)
    # display(styler_calculus)

In [2]:
def formulas_contain_items_on_formula_sheet(formulas, formula_sheet_list):
    """Returns true if there are one or more items in formulas that 
    are contained in  formula_sheet_list"""
    if formulas is None or formula_sheet_list is None:
        return (False)
    else:
        formulas_ex_null = formulas.loc[lambda x: x.notnull()]
        return(len(
            (set(formulas_ex_null) & set(formula_sheet_list))
            )>0)


def get_formula_display_string(input_series, **kwarg):
    """Returns a summmary formula table in markdown format with embedded
    html.  Input series is a pandas series with Indices State, Subect code
    and category.  **Kwarg must be called with parameter  = df_formulas
    where df_Formulas contains fields State, Subject code, Category, 
    Formula_1, Formula_2"""
    
    df_formulas = kwarg['df_formulas']
    df = df_formulas[(
        (df_formulas['State'] == str(input_series['State'])) &
        (df_formulas['Formula sub category 2'] == str(
            input_series['Formula sub category 2'])) &
        (df_formulas['Subject code'] == str(input_series['Subject code'])) & 
        (df_formulas['Category'] == str(input_series['Category'])))]

    formula_1_and_2 = pd.concat([df['Formula_1'], df['Formula_2']])
    formula_sheet_list =kwarg.get('formula_sheet_list')
    
    formula_2_col_is_empty = df['Formula_2'].dropna().empty    
    if formula_2_col_is_empty:
        df = df[['Formula_1']]
    else:
        df = df[['Formula_1', 'Formula_2']]

    output_string='#  \n<br>\n'
                      
    output_string+=(formulas.df_to_formula_styled_table(
        df=df, col_widths={'Formula_1':300, 'Formula_2':400},
        display_col_headers = False)).to_html()

    if formulas_contain_items_on_formula_sheet(
        formula_1_and_2, formula_sheet_list):

        output_string = (
            '{{< tabs "uniqueid" >}}\n\n' + 
            '{{< tab "Standard view" >}}\n\n' + 
            output_string + '{{< /tab >}}')
                         
        output_string+='\n\n' + '{{< tab "Highlight items on formula sheet" >}}'
        
        cols_to_highlight_if_in_formula_sheet = (
            kwarg.get('cols_to_highlight_if_in_formula_sheet'))
        
        output_string+=(formulas.df_to_formula_styled_table(
            df=df, col_widths={'Formula_1':300, 'Formula_2':400},
            cols_to_highlight_if_in_formula_sheet = (
                cols_to_highlight_if_in_formula_sheet),
            formula_sheet_list = formula_sheet_list,
            display_col_headers = False)).to_html()

        output_string+='{{< /tab >}}\n{{< /tabs >}}'
                      
    return(output_string)